<center>
    <img src="http://sct.inf.utfsm.cl/wp-content/uploads/2020/04/logo_di.png" style="width:60%">
    <h1> INF285 - Computación Científica </h1>
    <h2> Laboratorio 2</h2>
    <h2> [S]cientific [C]omputing [T]eam </a> </h2>
    <h2> 2025-2</h2>
</center>

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import time
from ipywidgets import interact

In [11]:
# No modificar este código
def solve_triangular(A, b, upper=True):
    n = b.shape[0]
    x = np.zeros_like(b)
    if upper==True:
        #perform backward-substitution
        x[-1] = (1./A[-1,-1]) * b[-1]
        for i in range(n-2, -1, -1):
            x[i] = (1./A[i,i]) * (b[i] - np.sum(A[i,i+1:] * x[i+1:]))
    else:
        #perform forward-substitution
        x[0] = (1./A[0,0]) * b[0]
        for i in range(1,n):
            x[i] = (1./A[i,i]) * (b[i] - np.sum(A[i,:i] * x[:i]))
    return x


#permutation between rows i and j on matrix A
def row_perm(A, i, j):
    tmp = np.copy(A[i])
    A[i] = A[j]
    A[j] = tmp


def palu_decomp(A, show=False, print_precision=2):
    N,_ = A.shape
    P = np.identity(N)
    L = np.zeros((N,N))
    U = np.copy(A)
    if show:
        print('Initial matrices')
        print('P = '); print(np.array_str(P, precision=print_precision, suppress_small=True))
        print('L = '); print(np.array_str(L+np.eye(N), precision=print_precision, suppress_small=True))
        print('U = '); print(np.array_str(U, precision=print_precision, suppress_small=True))
        print('----------------------------------------')
    #iterating through columns
    for j in range(N-1):
        #determine the new pivot
        p_index = np.argmax(np.abs(U[j:,j]))
        if p_index != 0:
            row_perm(P, j, j+p_index)
            row_perm(U, j, j+p_index)
            row_perm(L, j, j+p_index)
            if show:
                print('A permutation has been made')
                print('P = '); print(np.array_str(P, precision=print_precision, suppress_small=True))
                print('L = '); print(np.array_str(L+np.eye(N), precision=print_precision, suppress_small=True))
                print('U = '); print(np.array_str(U, precision=print_precision, suppress_small=True))
                print('----------------------------------------')
        #iterating through rows
        for i in range(j+1,N):
            L[i,j] = U[i,j]/U[j,j]
            U[i] -= L[i,j]*U[j]
            if show:
                print('P = '); print(np.array_str(P, precision=print_precision, suppress_small=True))
                print('L = '); print(np.array_str(L+np.eye(N), precision=print_precision, suppress_small=True))
                print('U = '); print(np.array_str(U, precision=print_precision, suppress_small=True))
                print('----------------------------------------')
    np.fill_diagonal(L,1)
    return P,L,U


def solve_palu(A, b, show=False, print_precision=2):
    P,L,U = palu_decomp(A, show, print_precision=print_precision)
    # A.x = b -> P.A.x = P.b = b' -> L.U.x = b'
    b = np.dot(P,b)
    # L.c = b' with c = U.x
    c = solve_triangular(L, b, upper=False)
    x = solve_triangular(U, c)
    return x

## Funciones

Para la implementación de las preguntas, considere que **solo** tiene a su disposición las siguientes funciones:

- **np.array(x)**: Convierte **x** en un arreglo de NumPy.

- **np.zeros(n)**: Crea un arreglo de NumPy con **n** ceros.

- **np.ones(n)**: Crea un arreglo de NumPy con **n** unos.

- **np.sin(x)**: Devuelve $\sin(x)$, donde **x** puede ser un escalar o un vector.

- **np.cos(x)**: Devuelve $\cos(x)$, donde **x** puede ser un escalar o un vector.

- **np.power(x, n)**: Evalúa la expresión $x^n$ si **x** y **n** son escalares. En caso de que **x** e **n** sean vectores, deben tener la misma dimensión y entrega la evaluación elemento a elemento. Si solo uno de los términos es un vector, entrega el vector donde el término constante se consideró para cada término de vector.

- **np.dot(x, y)**: Si ambas variables son vectores devuelve su producto punto, si una es una matriz y la otra un vector devuelve su producto matriz-vector, si ambas son matrices devuelve su producto matricial.

- **np.diag(A, {k=-1,k=0,k=1})**: Si **A** es una matriz, retorna el vector con sus elementos de la diagonal indicada por **k** (subdiagonal, diagonal principal y supradiagonal). Si **A** es un vector, crea una matriz diagonal con esos elementos colocados en la diagonal desplazada correspondiente.

- **np.concatenate(([a], x))**: Concatena el escalar **a** al inicio del vector **x**, notar que los doble parentesis son necesarios.

- **np.concatenate((x, [b]))**: Concatena el escalar **b** al final vector **x**, notar que los doble parentesis son necesarios.

- **solve_triangular(A, b, upper)**: Resuelve el sistema $A\vec{x} = \vec{b}$ para $A$ triangular. Usa forward-substitution si `upper=False`, y backward-substitution si `upper=True`.

- **palu_decomp(A)**: Calcula la descomposición $PA = LU$ de la matriz $A$, devuelve las matrices $P$, $L$ y $U$ tales que $A = P^{T}LU$. Con $P$ la matriz de permutación transpuesta, $L$ una matriz triangular inferior, y $U$ una matriz triangular superior.

- **np.linalg.norm(x, ord=2)**: Siendo **x** un vector, devuelve la norma 2 de **x**, osea devuelve $\|x\|_2$.

- **len(x)**: Devuelve el largo de **x**.

- **range(a, b)**: Crea un objeto iterable que produce los enteros desde **a** hasta **b-1** (incluye **a**, excluye **b**).

## Introducción

El péndulo no lineal amortiguado es un sistema fundamental para el estudio de fenómenos dinámicos donde intervienen fuerzas restauradoras no lineales y disipación. La ecuación diferencial que lo describe incluye tanto el efecto de la gravedad, que depende del ángulo a través de la función seno, como la fricción, que introduce amortiguamiento en el movimiento.

En muchas aplicaciones es relevante analizar el comportamiento del péndulo cuando se imponen restricciones en los valores del ángulo al inicio y al final de un intervalo de tiempo. Este tipo de planteamiento, conocido como problema de valores frontera (BVP), permite modelar trayectorias bajo condiciones específicas.

Se desea determinar la evolución angular $\theta (t)$ de un péndulo de longitud $L$, sometido a amortiguamiento y gravedad, cumpliendo condiciones de frontera especificadas al inicio y al final del intervalo temporal:

$$
L\frac{d^2\theta}{dt^2} + b\frac{d\theta}{dt} + g \sin (\theta) = 0, \quad t \in [0,T],
$$

$$
\theta (0)=\theta_0,\; \theta (T)=\theta_T.
$$

Aquí, $b$ controla la fuerza disipativa, $g$ es la aceleración gravitacional, y los valores de $\theta_0$, $\theta_T$ fijan los ángulos en los extremos.

Para resolver el sistema, se divide el intervalo temporal en $n+1$ puntos equidistantes, usando diferencias finitas para aproximar derivadas. Así, cada punto $t_i$ se asocia a un valor desconocido $\theta_i$, excepto en los extremos donde los valores son impuestos por las condiciones de frontera.

Para cada nodo interno $(i = 1, \ldots, n-1)$, la ecuación discreta es:

$$
\mathbf{F}_i(\boldsymbol{\theta}) = \frac{L}{h^2}(\theta_{i-1} - 2\theta_i + \theta_{i+1}) + \frac{b}{2h}(\theta_{i+1} - \theta_{i-1}) + g \sin (\theta_{i})
$$

Resolver este sistema implica encontrar los valores de $\theta_i$ que satisfacen todas las ecuaciones simultáneamente.

## Preguntas

Al juntar todas las ecuaciones en el sistema $\mathbf{F}(\mathbf{\theta}) = \mathbf{0}$ y aplicando las condiciones de frontera, se obtiene la siguiente función vector:

$$
\begin{align*}
   \mathbf{F}(\boldsymbol{\theta}) &=
    \begin{bmatrix}
       \dfrac{L}{h^2}(\theta_{0} - 2\theta_1 + \theta_{2}) + \dfrac{b}{2h}(\theta_{2} - \theta_{0}) + g \sin (\theta_{1})\\[0.8em]
       \dfrac{L}{h^2}(\theta_{1} - 2\theta_2 + \theta_{3}) + \dfrac{b}{2h}(\theta_{3} - \theta_{1}) + g \sin (\theta_{2})\\[0.8em]
       \vdots\\[0.8em]
       \dfrac{L}{h^2}(\theta_{i-1} - 2\theta_i + \theta_{i+1}) + \dfrac{b}{2h}(\theta_{i+1} - \theta_{i-1}) + g \sin (\theta_{i})\\[0.8em]
       \vdots\\[0.8em]
       \dfrac{L}{h^2}(\theta_{n-3} - 2\theta_{n-2} + \theta_{n-1}) + \dfrac{b}{2h}(\theta_{n-1} - \theta_{n-3}) + g \sin (\theta_{n-2})\\[0.8em]
       \dfrac{L}{h^2}(\theta_{n-2} - 2\theta_{n-1} + \theta_{T}) + \dfrac{b}{2h}(\theta_{T} - \theta_{n-2}) + g \sin (\theta_{n-1})\\[0.8em]
    \end{bmatrix}
\end{align*}
$$

1. **(25 puntos)** Implemente la función $\mathbf{F}$ en código, tal que, dado un vector de valores de $\theta$ la función devuelva $\mathbf{F}(\boldsymbol{\theta})$. Con el objetivo de que se pueda ocupar en el método de *Newton*.

    *Hint: It might be better to concatenate $\theta_0$ and $\theta_T$ to the array $\boldsymbol{\theta}$ before computing $\mathbf{F}(\boldsymbol{\theta})$.*


In [6]:
def F(theta, L, b, g, h, theta0, thetaT):
  """
  input:
  theta  : (ndarray) Vector of theta values.
  L      : (float) Length of the pendulum.
  b      : (float) Damping coefficient.
  g      : (float) Gravity acceleration.
  h      : (float) Finite difference constant.
  theta0 : (float) Boundary constant at theta(0).
  thetaT : (float) Boundary constant at theta(T).

  output:
  f      : (ndarray) Vector of theta values evaluated on F.
  """
  ###### acá va su código ######
  n=len(theta)
  th=np.zeros(n + 2)
  th[0] = theta0
  th[ n + 1] = thetaT
  th[1:n+1] = theta
  invh= 1.0/h
  inv2h= invh*invh
  d2 = (th[2:n+2] - 2.0 * th[1:n+1] + th[0:n]) * inv2h
  d1 = (th[2:n+2] - th[0:n]) * (0.5 * invh)
  f = d2 + (b / L) * d1 + (g / L) * np.sin(th[1:n+1])


  ###### acá va su código ######
  return f

(4,)
[  1.07836748   2.04695944   2.99609803 -46.33370025]


2. **(25 puntos)**
Implemente la función para calcular el jacobiano $\mathbf{J}$ en código, tal que, dado un vector de valores de $\theta$ la función devuelva $\mathbf{J}(\boldsymbol{\theta})$. Con el objetivo de que se pueda ocupar en el método de *Newton*.

In [7]:
def JacF(theta, L, b, g, h):
  """
  input:
  theta  : (ndarray) Vector of theta values.
  L      : (float) Length of the pendulum.
  b      : (float) Damping coefficient.
  g      : (float) Gravity acceleration.
  h      : (float) Finite difference constant.

  output:
  J     : (ndarray) Jacobian of F evaluated on theta values.
  """
  ###### acá va su código ######
  N    = len(theta)
  invh = 1.0 / h
  invh2 = invh * invh
  d1   = (b / L) * (0.5 * invh)
  diag = (g / L) * np.cos(theta) - 2.0 * invh2
  sub  = (np.ones(N-1) * (invh2 - d1)) if N > 1 else np.zeros(0)
  sup  = (np.ones(N-1) * (invh2 + d1)) if N > 1 else np.zeros(0)
  J = np.diag(diag)
  if N > 1:
    J = J + np.diag(sub, k=-1) + np.diag(sup, k=+1)

  ###### acá va su código ######
  return J



(5, 5)
(5,)


Ahora que tenemos tanto $\mathbf{F}(\boldsymbol{\theta})$ como el jacobiano $\mathbf{J}(\boldsymbol{\theta})$, podemos ocupar el método iterativo de *Newton* para $\mathbb{R}^n$ para encontrar la solución del BVP del péndulo.

3. **(25 puntos)** Implemente la función `QuasiNewton(theta, L, b, g, h, theta0, thetaT, iterNewton=30, iterPALU=1)` que resuelva el problema usando un método tipo Newton para $\mathbb{R}^n$.

La diferencia de **QuasiNewton** con el método de Newton estándar es que, en cada iteración, en vez de aplicar PALU una sola vez, se pueda aplicar PALU una cantidad `iterPALU` de veces, para evitar tener que recalcular $\mathbf{J}$. Notar que si `iterPALU = 1`, el método coincide con Newton estándar.

La función debe devolver la aproximación final de la BVP, y el vector de errores obtenidos en cada iteración de Newton (no de PALU).

*Hint: The error is the 2-norm of the correction vector.*

In [8]:
def QuasiNewton(theta, L, b, g, h, theta0, thetaT, iterNewton=30, iterPALU=1, delta=1e-12):
  """
  input:
  theta      : (ndarray) Vector of guess values of theta.
  L          : (float) Length of the pendulum.
  b          : (float) Damping coefficient.
  g          : (float) Gravity acceleration.
  h          : (float) Finite difference constant.
  theta0     : (float) Boundary constant at theta(0)
  thetaT     : (float) Boundary constant at theta(T)
  iterNewton : (int) Maximum number of iterations of Newton.
  iterPALU   : (int) Maximum number of iterations of PALU per iteration of Newton.
  delta      : (float) Threshold tolerance for stopping criteria, i.e. if ||F(theta)||_2<tol, stop.

  output:
  theta_f : (ndarray) Theta values solutions of the BVP using Newton Rn (Includes theta0 and thetaT).
  errors  : (ndarray) Errors at each iteration.
  """
  ###### acá va su código ######
  errs_list = []
  P = None
  Lm = None
  Um = None
  for k in range(iterNewton):
      r  = F(theta, L, b, g, h, theta0, thetaT)
      rn = np.linalg.norm(r, ord=2)
      errs_list.append(rn)
      if rn < delta:
          break


      if (k % iterPALU) == 0 or (P is None):
        J = JacF(theta, L, b, g, h)     # matriz NxN
        P, Lm, Um = palu_decomp(J)      # J = P^T L U


        Pb = np.dot(P, -r)                          # P(-r)
        y  = solve_triangular(Lm, Pb, upper=False)
        w  = solve_triangular(Um, y,  upper=True)

        theta = theta + w


      theta_f = np.concatenate(([theta0], theta, [thetaT]))

      errors = np.array(errs_list)

  ###### acá va su código ######
  return theta_f, errors

4. **(25 puntos)** Explique si existe un valor de `iterPALU` $> 1$ que permita alcanzar un error final y un tiempo total de ejecución comparable o menor al método de Newton estándar. Notar que se ha definido un _initial guess_ conveniente, es decir el vector `-np.sin(np.linspace(0,2*np.pi,N))*beta`. Su análisis debe considerar el efecto tanto de `iterPALU` como del coeficiente $\beta$. Hint: $\beta$ close to the unity may be interesting.

---
**Respuesta:**
 Con el *initial guess* $theta^{(0)}=\sin(\text{linspace}(0,2\pi,N))\cdot\beta$ y $\beta\approx 1$, el Jacobiano cambia poco entre iteraciones, por lo que es posible congelarlo durante varios pasos $\text{iterPALU}>1$ sin perder precisión en el error final.
 Al ver el gráfico vemos que para  $\text{iterPALU}=1$ ambos métodos se comportan de la misma forma, tienen la misma convergencia y el mismo error
 Con valores intermedios, como `iterPALU = 2–3`, se obtiene un error final comparable al estándar pero con menos factorizaciones PALU, lo que reduce el costo total.  
 Para `iterPALU` demasiado grandes ≥10, la convergencia se degrada y ya no conviene.  
-En cuanto a $beta$, si se elige cercano a 1 se obtiene la convergencia más rápida; si se reduce demasiado, el método necesita más pasos.

Con esto podemos decir que para un iterPalu pequeño (2-3) pero con un $beta$ cercano a 1, el método de Quasi-Newton mantiene la misma precisión que el método de Newton clásico pero tiene un menor costo computacional
---

In [9]:
# No modificar este código
L = 1.0
b = 0.2
g = 9.81
T = 2
N = 35
h = T/(N+1)

theta0 = 0.1
thetaT = 0.5

In [13]:
def show_comparison(iterPALU_exp=1, beta=0.0):

    # Initial guess
    theta_guess = -np.sin(np.linspace(0,2*np.pi,N))*beta

    # Newton Clásico (iterPALU = 1)
    start = time.time()
    theta_newton, errors_newton = QuasiNewton(theta_guess.copy(), L, b, g, h, theta0, thetaT)
    t_newton = time.time() - start

    # Quasi-Newton
    start = time.time()
    theta_qn, errors_qn = QuasiNewton(theta_guess.copy(), L, b, g, h, theta0, thetaT, iterNewton=30, iterPALU=iterPALU_exp)
    t_qn = time.time() - start

    # Gráfico de convergencia
    plt.figure(figsize=(10, 5))
    plt.subplot(121)
    plt.plot(errors_newton, 'bo--', label=f"Newton clásico (iterPALU=1), t={t_newton:.3f}s")
    plt.plot(errors_qn, 'rx--', label=f"Quasi-Newton (iterPALU={iterPALU_exp}), t={t_qn:.3f}s")
    plt.yscale("log")
    plt.xlabel("Iteración de Newton")
    plt.ylabel(r"$||w||_2$")
    plt.title(f"Comparación de convergencia:\n Newton clásico vs Quasi-Newton")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()

    plt.subplot(122)
    t = np.linspace(0, T, N+2)

    data = np.load('pendulum_solution.npz')
    t_bvp = data['t']
    theta = data['theta']

    plt.plot(t_bvp, theta, 'g-', linewidth=2, label=r'$\theta(t)$ (Solución exacta)', alpha=0.5)
    plt.scatter(t, theta_newton, marker='o', color='blue', s=30, label=r'$\theta(t)$ (Newton)', alpha=0.5)
    plt.scatter(t, theta_qn, marker='s', color='red', s=30, label=r'$\theta(t)$ (Quasi-Newton)', alpha=0.5)


    plt.xlabel('t')
    plt.ylabel(r'$\theta(t)$')
    plt.title(f"Péndulo amortiguado:\n Comparación de soluciones")
    plt.grid(True, alpha=0.3)
    plt.legend()
    plt.tight_layout()

    plt.show()

interact(show_comparison,iterPALU_exp=(1,20,1),beta=(-2,2,0.1))

interactive(children=(IntSlider(value=1, description='iterPALU_exp', max=20, min=1), FloatSlider(value=0.0, de…

<function __main__.show_comparison(iterPALU_exp=1, beta=0.0)>

---

### Felicidades, llegaste al final del laboratorio recuerda subir el archivo al Aula y con el nombre `apellido1_apellido2_lab_2`